## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
pd.options.mode.chained_assignment = None

In [38]:
df = pd.read_csv('data/aviation_data.csv')

/Users/jacklocke/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [39]:
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.9222,-81.8781,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [40]:
#First, we are going to remove a lot of columns that have too many null values or we know we wont be using/investigating
#for our analysis. 
df.isna().sum()

Event.Id                   1459
Investigation.Type            0
Accident.Number            1459
Event.Date                 1459
Location                   1511
Country                    1685
Latitude                  55966
Longitude                 55975
Airport.Code              40099
Airport.Name              37558
Injury.Severity            2459
Aircraft.damage            4653
Aircraft.Category         58061
Registration.Number        2776
Make                       1522
Model                      1551
Amateur.Built              1561
Number.of.Engines          7543
Engine.Type                8536
FAR.Description           58325
Schedule                  77766
Purpose.of.flight          7651
Air.carrier               73700
Total.Fatal.Injuries      12860
Total.Serious.Injuries    13969
Total.Minor.Injuries      13392
Total.Uninjured            7371
Weather.Condition          5951
Broad.phase.of.flight     28624
Report.Status              7840
Publication.Date          16689
dtype: i

In [41]:
df = df.drop(columns=['Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 'Registration.Number', 'Schedule', 
                      'FAR.Description','Publication.Date','Event.Id','Accident.Number','Injury.Severity','Report.Status',
                      'Publication.Date'])
df.head()

,Investigation.Type,Event.Date,Location,Country,Aircraft.damage,Aircraft.Category,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight
0,Accident,1948-10-24,"MOOSE CREEK, ID",United States,Destroyed,NaN,Stinson,108-3,No,1.0,Reciprocating,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise
1,Accident,1962-07-19,"BRIDGEPORT, CA",United States,Destroyed,NaN,Piper,PA24-180,No,1.0,Reciprocating,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown
2,Accident,1974-08-30,"Saltville, VA",United States,Destroyed,NaN,Cessna,172M,No,1.0,Reciprocating,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise
3,Accident,1977-06-19,"EUREKA, CA",United States,Destroyed,NaN,Rockwell,112,No,1.0,Reciprocating,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise
4,Accident,1979-08-02,"Canton, OH",United States,Destroyed,NaN,Cessna,501,No,NaN,NaN,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach


In [42]:
#Im going to lowercase all the columns to make them easier to deal with but i like the period seperation. 
df.columns = df.columns.str.lower()

In [43]:
df.columns

Index(['investigation.type', 'event.date', 'location', 'country',
       'aircraft.damage', 'aircraft.category', 'make', 'model',
       'amateur.built', 'number.of.engines', 'engine.type',
       'purpose.of.flight', 'air.carrier', 'total.fatal.injuries',
       'total.serious.injuries', 'total.minor.injuries', 'total.uninjured',
       'weather.condition', 'broad.phase.of.flight'],
      dtype='object')

In [44]:
#Before we start to deal with missing values i will investigate some of the columns further to get a better 
#understanding and see if we can start to narrow down our dataset. We already know we are going to want airplanes
#within the US. (will also be checking null/missing values before too to make sure its not too significant before
#doing anything) But in order to handle all the null/missing values in aircraft.category we will need to match the
#main specific airplane makes to them so we can fill in the null values appropiately so we can keep more airplane data.

In [45]:
#Doesn't appear to be a difference between accident as incident as both have fatalities/injuries. For now we will
#just convert everything to accident as it will be a good column to use for count later on.
df['investigation.type'].value_counts()

Accident      85015
Incident       3874
25-09-2020      702
26-09-2020       60
02-02-2021       39
              ...  
04-08-2021        1
06-01-2021        1
05-01-2021        1
05-04-2021        1
11-02-2021        1
Name: investigation.type, Length: 71, dtype: int64

In [46]:
df[df['investigation.type']=='Incident']['total.fatal.injuries'].value_counts()

0.0    3210
1.0      17
3.0       1
2.0       1
Name: total.fatal.injuries, dtype: int64

In [47]:
df['accidents'] = df['investigation.type'].apply(lambda x: 'Accident')

In [48]:
df.drop(columns='investigation.type',inplace=True)

In [49]:
df.head()

,event.date,location,country,aircraft.damage,aircraft.category,make,model,amateur.built,number.of.engines,engine.type,purpose.of.flight,air.carrier,total.fatal.injuries,total.serious.injuries,total.minor.injuries,total.uninjured,weather.condition,broad.phase.of.flight,accidents
0,1948-10-24,"MOOSE CREEK, ID",United States,Destroyed,NaN,Stinson,108-3,No,1.0,Reciprocating,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Accident
1,1962-07-19,"BRIDGEPORT, CA",United States,Destroyed,NaN,Piper,PA24-180,No,1.0,Reciprocating,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Accident
2,1974-08-30,"Saltville, VA",United States,Destroyed,NaN,Cessna,172M,No,1.0,Reciprocating,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Accident
3,1977-06-19,"EUREKA, CA",United States,Destroyed,NaN,Rockwell,112,No,1.0,Reciprocating,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Accident
4,1979-08-02,"Canton, OH",United States,Destroyed,NaN,Cessna,501,No,NaN,NaN,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Accident


In [50]:
#HOLD UP on this one, might be good to explore cause could be good business rec in general. 
#I did some research on the date of airplanes being flown. The website is from a private business that sells private
#jets. They essentially state that the age of the aircraft is not a concern but more so the maintenance on the aircraft.
#They found the standard aircraft to be about 10-20 years old and they do not recommend aircrafts to clients that are
#30+ years of age. Considering our dataset is based on accidents. The maintenance/age could be a factor. We will then
#focus on accidents only recorded since 1995 which falls within that 30 year range but is also generous. 
#(https://www.paramountbusinessjets.com/faq/age-of-aircraft-safety-factor)

In [51]:
#df['event.date'] = df['event.date'].apply(lambda x: int(x[0:4]))

In [52]:
#df = df[df['event.date']>=1995]

In [53]:
#I only want to look at accidents in the US so for country and location i will narrow it down and then remove those
#columns as im not going to use any form of location for my analysis. 

In [54]:
df = df[df['country']=='United States']

In [55]:
df.drop(columns=['location','country'],inplace=True)

In [56]:
df

,event.date,aircraft.damage,aircraft.category,make,model,amateur.built,number.of.engines,engine.type,purpose.of.flight,air.carrier,total.fatal.injuries,total.serious.injuries,total.minor.injuries,total.uninjured,weather.condition,broad.phase.of.flight,accidents
0,1948-10-24,Destroyed,NaN,Stinson,108-3,No,1.0,Reciprocating,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Accident
1,1962-07-19,Destroyed,NaN,Piper,PA24-180,No,1.0,Reciprocating,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Accident
2,1974-08-30,Destroyed,NaN,Cessna,172M,No,1.0,Reciprocating,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Accident
3,1977-06-19,Destroyed,NaN,Rockwell,112,No,1.0,Reciprocating,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Accident
4,1979-08-02,Destroyed,NaN,Cessna,501,No,NaN,NaN,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Accident
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90343,2022-12-26,NaN,NaN,PIPER,PA-28-151,No,NaN,NaN,Personal,NaN,0.0,1.0,0.0,0.0,NaN,NaN,Accident
90344,2022-12-26,NaN,NaN,BELLANCA,7ECA,No,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,Accident
90345,2022-12-26,Substantial,Airplane,AMERICAN CHAMPION AIRCRAFT,8GCBC,No,1.0,NaN,Personal,NaN,0.0,0.0,0.0,1.0,VMC,NaN,Accident
90346,2022-12-26,NaN,NaN,CESSNA,210N,No,NaN,NaN,Personal,MC CESSNA 210N LLC,0.0,0.0,0.0,0.0,NaN,NaN,Accident


In [57]:
#Common sense, we are not going to suggest an aircraft that was built by an amateur so we will filter out any 'Yes'
#values and then drop the column. 

In [58]:
df = df[df['amateur.built']=='No']

In [59]:
df.drop(columns='amateur.built',inplace=True)

In [60]:
df.head()

,event.date,aircraft.damage,aircraft.category,make,model,number.of.engines,engine.type,purpose.of.flight,air.carrier,total.fatal.injuries,total.serious.injuries,total.minor.injuries,total.uninjured,weather.condition,broad.phase.of.flight,accidents
0,1948-10-24,Destroyed,NaN,Stinson,108-3,1.0,Reciprocating,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Accident
1,1962-07-19,Destroyed,NaN,Piper,PA24-180,1.0,Reciprocating,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Accident
2,1974-08-30,Destroyed,NaN,Cessna,172M,1.0,Reciprocating,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Accident
3,1977-06-19,Destroyed,NaN,Rockwell,112,1.0,Reciprocating,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Accident
4,1979-08-02,Destroyed,NaN,Cessna,501,NaN,NaN,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Accident


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73906 entries, 0 to 90347
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   event.date              73906 non-null  object 
 1   aircraft.damage         71949 non-null  object 
 2   aircraft.category       24697 non-null  object 
 3   make                    73897 non-null  object 
 4   model                   73891 non-null  object 
 5   number.of.engines       72237 non-null  float64
 6   engine.type             71156 non-null  object 
 7   purpose.of.flight       71481 non-null  object 
 8   air.carrier             13518 non-null  object 
 9   total.fatal.injuries    64425 non-null  float64
 10  total.serious.injuries  63850 non-null  float64
 11  total.minor.injuries    64443 non-null  float64
 12  total.uninjured         69830 non-null  float64
 13  weather.condition       73285 non-null  object 
 14  broad.phase.of.flight   55594 non-null

In [62]:
#The rest of the columns id like to explore further and handle missing values, but it appears that air.carrier has
#an extreme amount of missing values which is not going to be helpful for us. We will have to drop this column.

In [63]:
df.drop(columns='air.carrier',inplace=True)

In [64]:
df.head()

,event.date,aircraft.damage,aircraft.category,make,model,number.of.engines,engine.type,purpose.of.flight,total.fatal.injuries,total.serious.injuries,total.minor.injuries,total.uninjured,weather.condition,accidents
0,1948-10-24,Destroyed,NaN,Stinson,108-3,1.0,Reciprocating,Personal,2.0,0.0,0.0,0.0,UNK,Accident
1,1962-07-19,Destroyed,NaN,Piper,PA24-180,1.0,Reciprocating,Personal,4.0,0.0,0.0,0.0,UNK,Accident
2,1974-08-30,Destroyed,NaN,Cessna,172M,1.0,Reciprocating,Personal,3.0,NaN,NaN,NaN,IMC,Accident
3,1977-06-19,Destroyed,NaN,Rockwell,112,1.0,Reciprocating,Personal,2.0,0.0,0.0,0.0,IMC,Accident
4,1979-08-02,Destroyed,NaN,Cessna,501,NaN,NaN,Personal,1.0,2.0,NaN,0.0,VMC,Accident


In [65]:
#Now we will handle the aircraft.category missing values. With the largest counts of make. 

In [66]:
df['make'].isna().sum()

9

In [67]:
df['make'] = df['make'].fillna('Cessna')

In [68]:
df['make'] = df['make'].apply(lambda x: x.lower())

In [69]:
df['make'].value_counts()

cessna                   25779
piper                    14125
beech                     5043
bell                      2272
boeing                    1481
                         ...  
slingsby aviation plc        1
cessna soloy                 1
santa fe aircraft llc        1
yowell                       1
phantom                      1
Name: make, Length: 1771, dtype: int64

In [72]:
df['aircraft.category'] = df.apply(lambda x: 'Airplane' if x['make'] in ['cessna','piper','beech','mooney','boeing']else x, axis=1)

In [85]:
df = df[df['aircraft.category']=='Airplane']

In [300]:
#Now we can begin to explore our dataset and handle missing/null values to help come up with who our specific 
#stakeholders will be

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47713 entries, 1 to 90347
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   event.date              47713 non-null  object 
 1   aircraft.damage         46857 non-null  object 
 2   aircraft.category       47713 non-null  object 
 3   make                    47713 non-null  object 
 4   model                   47705 non-null  object 
 5   number.of.engines       47352 non-null  float64
 6   engine.type             46647 non-null  object 
 7   purpose.of.flight       46577 non-null  object 
 8   total.fatal.injuries    41909 non-null  float64
 9   total.serious.injuries  41353 non-null  float64
 10  total.minor.injuries    41740 non-null  float64
 11  total.uninjured         45227 non-null  float64
 12  weather.condition       47391 non-null  object 
 13  accidents               47713 non-null  object 
dtypes: float64(5), object(9)
memory usage: